### Imports

In [150]:
import pandas as pd 
import re
import numpy as np 
import matplotlib.pyplot as plt
import requests
from collections import defaultdict

from pyproj import CRS
import folium
import branca.colormap as cm

## Code using `requests`

In [149]:
WAY_CACHE = {}

def get_way(way_id):
    if way_id in WAY_CACHE:
        return WAY_CACHE[way_id]
    
    OVERPASS_URL = "http://overpass-api.de/api/interpreter"
    query = f'''
    [out:json];
    way({way_id});
    out geom;
    '''
    response = requests.post(OVERPASS_URL, data={"data": query})
    data = response.json()
    way = data['elements'][0]
    WAY_CACHE[way_id] = way
    
    return way

### Data

In [57]:
tdf = pd.read_csv('data/train-1500.csv')
mdf = pd.read_csv('data/matched_results_1500_updated.csv')
osm_edges = pd.read_csv('data/edges.csv')

In [58]:
for col in mdf.columns:
    if col not in ['idx', 'id', 'match_geom', 'match_pt']:
        mdf[col] = mdf[col].apply(eval)

In [5]:
osm_edges.head(2)

,u,v,key,osmid,oneway,lanes,highway,maxspeed,reversed,length,ref,bridge,name,access,width,junction,tunnel,fid,geometry
0,25503936,4722746638,0,479127843,True,2,motorway_link,70,False,32.388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"LINESTRING (-8.6406364 41.1660713, -8.6409114 ..."
1,25503936,281726624,0,"[4256507, 1175924414]",True,1,motorway_link,40,False,223.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"LINESTRING (-8.6406364 41.1660713, -8.6407202 ..."


In [6]:
mdf.head(2)

,idx,id,match_path,match_edge_by_pt,match_edge_by_idx,match_geom,match_pt,edge_id,source,target,error,length,offset,spdist,ep,tp
0,0,1372636858620000589,"[4011, 11074, 4010, 9279, 9277, 576, 268, 4056...","[4011, 4011, 9277, 9277, 576, 576, 268, 268, 2...","[0, 0, 4, 4, 5, 5, 6, 6, 10, 11, 12, 13, 13, 1...","LINESTRING(-8.6188434 41.141753,-8.6186843 41....","LINESTRING(-8.6188434 41.141753,-8.6187842 41....","[4011, 4011, 9277, 9277, 576, 576, 268, 268, 2...","[475564016, 475564016, 1788784885, 1788784885,...","[2915001116, 2915001116, 122549700, 122549700,...","[0.00039544022304002633, 0.000505487040587886,...","[0.0005062938626436966, 0.0005062938626436966,...","[0.0003136062916914766, 0.00038535461428278146...","[0.0, 7.174832259130486e-05, 0.002071537387491...","[0.9968774260068927, 0.9949026926236646, 0.999...","[0.0, 1.0, 1.0, 0.9884877389173452, 0.99996221..."
1,1,1372637303620000596,"[7562, 14878, 13857, 13864, 13860, 2211, 9641,...","[7562, 7562, 2211, 7674, 324, 7641, 7653, 7643...","[0, 0, 5, 7, 8, 9, 12, 15, 16, 18, 20, 22, 22,...","LINESTRING(-8.6398592 41.159752,-8.6400962 41....","LINESTRING(-8.6398592 41.159752,-8.6403572 41....","[7562, 7562, 2211, 7674, 324, 7641, 7653, 7643...","[1239645082, 1239645082, 343646787, 1264279444...","[9026838533, 9026838533, 1842003078, 111830195...","[7.548733359574764e-05, 3.8372431021183606e-05...","[0.002341263160321466, 0.002341263160321466, 0...","[0.001679648540514599, 0.002184301328853114, 0...","[0.0, 0.0005046527883385151, 0.001860062043703...","[0.9998860397432955, 0.9999705515643642, 0.999...","[0.0, 1.0, 1.0, 1.0, 0.9999463264889296, 0.999..."


## Based on other people code: Find the top 10 FID

In [113]:
def linestring2list(string):
    if not isinstance(string, str):
        string = string.wkt

    assert string[:10] == 'LINESTRING'
    string = string.replace(', ', ',')
    string = string[string.find('(')+1:-1]

    return [[float(j) for j in i.split(' ')] for i in string.split(',')]

# givn the road fid, return the information of roads: (osmid, u, v, length)
def route_info(fid): 

    point_list = osm_edges.loc[fid, 'geometry']
    point_list = linestring2list(point_list)

    length = 0
    if len(point_list) > 1:
        for i in range(len(point_list)-1):
            length += ((point_list[i+1][0]-point_list[i][0]) ** 2 + (point_list[i+1][1] - point_list[i][1]) ** 2) ** 0.5
    length = round(length, 7)

    return (osm_edges.loc[fid, 'osmid'], int(osm_edges.loc[fid, 'u']), int(osm_edges.loc[fid, 'v']), length)

# two roads with different FID might be the same road (2 way road)
# while two roads with the same OSMID might be different(e.g. osmid=868082261), and one road might have multiple OSMID (e.g. fid=4)
def construct_road_fid2id(results): 

    road_fid2id_info = dict()
    road_uvd2id = dict()
    road_id2fids = dict()

    for result in results:
        cpath = result['cpath']
        opath = result['opath']
        for fid in cpath+opath:
            if not fid in road_fid2id_info:
                osmid, u, v, d = route_info(fid)
                uvd = (min(u,v), max(u,v), d)
                
                # len(route_uvd2id) increases every time a new uvd is added -> serve as a unique identifier for each unique uvd
                if uvd not in road_uvd2id:
                    road_id2fids[len(road_uvd2id)] = []
                    road_uvd2id[uvd] = len(road_uvd2id)
                ID = road_uvd2id[uvd]
                
                # Since there could be multiple fid associated with one road, we use a unique identifier based on uvd to group them together
                road_id2fids[ID].append(fid)
                road_fid2id_info[fid] = (ID, osmid, u, v, d)
                """
                ID is unique ID number of the road segment (self assigned)
                osmid is the osm ID of that road segment 
                u is start point of road segment
                v is end point of road segment
                d is length of road segment
                """

    return road_fid2id_info, road_id2fids


def most_often_fid(results, n=10):
    road_id2frequency = defaultdict(int)
    for route in results:
        repeat_c_ids = [road_fid2id_info[c][0] for c in route['cpath']]
        c_ids = []
        for r in repeat_c_ids:
            if len(c_ids) == 0 or r != c_ids[-1]:
                c_ids.append(r)

        for ID in c_ids:
            road_id2frequency[ID] += 1
    sorted_id = sorted(road_id2frequency.keys(), key=lambda x: road_id2frequency[x], reverse=True)

    result = []
    for i in range(n):
        ID = sorted_id[i]
        fids = tuple(road_id2fids[ID])
        road_name = osm_edges.loc[fids[0], "name"]
        result.append((fids, road_name, road_id2frequency[ID]))
    return result

In [59]:
all_results = []
ignore_number = 0
for n in range(1500):
    if len(mdf.loc[n, 'match_edge_by_pt']) < 2 or len(mdf.loc[n, 'match_path']) < 2:
        ignore_number += 1
        continue
    all_results.append(
        dict(t_number=mdf.loc[n, 'id'], 
             cpath=mdf.loc[n, 'match_path'], 
             opath=mdf.loc[n, 'match_edge_by_pt'], 
             offset=mdf.loc[n, 'offset'], 
             length=mdf.loc[n, 'length'],
             spdist=mdf.loc[n, 'spdist'], 
             mgeom=linestring2list(mdf.loc[n, 'match_geom'])
        ))

road_fid2id_info, road_id2fids = construct_road_fid2id(all_results)
method_1 = most_often(all_results)

In [148]:
method_1

[((9671, 64), 'Rua de Dom Manuel II', 152),
 ((29,), 'Rua Elísio de Melo', 146),
 ((63, 808), 'Rua de Dom Manuel II', 136),
 ((809, 9292), 'Rua de Dom Manuel II', 136),
 ((1019,), 'Rua do Doutor Magalhães Lemos', 134),
 ((2639, 811), 'Rua de Júlio Dinis', 128),
 ((812, 805), 'Rua de Júlio Dinis', 127),
 ((25,), 'Rua Elísio de Melo', 123),
 ((7871,), 'Rua de Pinto Bessa', 122),
 ((7860,), 'Túnel dos Almadas', 121)]

## Using Brian Method

In [124]:
way_id_freq = defaultdict(lambda: 0)

for match in mdf.match_path:
    for eid in match:
        e = osm_edges.iloc[eid]
        for way_id in to_int(e.osmid):
            way_id_freq[way_id] += 1

In [146]:
sorted_way_id = sorted(way_id_freq.items(), key=lambda item: item[1], reverse=True)
method_2 = sorted_way_id[:10]

In [147]:
method_2

[(14326493, 338),
 (453761206, 288),
 (224646232, 264),
 (704509573, 236),
 (35171965, 219),
 (14334234, 201),
 (12301715, 199),
 (35331293, 198),
 (1014684308, 196),
 (207634521, 194)]

## Visualisation using Folium for method 1

In [240]:
trip_coords = []
for road in method_1:
    fid = road[0][0]
    trip_coords.append(linestring2list(osm_edges.loc[fid, 'geometry']))

In [241]:
trip_coords[0][:4]

[[-8.6218374, 41.1475571],
 [-8.6220608, 41.1476127],
 [-8.6221516, 41.1476373],
 [-8.6224892, 41.1477231]]

In [242]:
# Bounding box for map view
all_points = [point for trip in trip_coords for point in trip]
lons, lats = zip(*all_points)

# Set map boundaries with margin (consistent with Code 2)
margin = 0.005
lon_min, lon_max = min(lons) - margin, max(lons) + margin
lat_min, lat_max = min(lats) - margin, max(lats) + margin

# Calculate center for consistent centering across both codes
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

# Initialize map and set bounds
fmap = folium.Map(location=[center_lat, center_lon], zoom_start=13, width=1500, height=1000)
fmap.fit_bounds([[lat_min, lon_min], [lat_max, lon_max]])
# Plot trips with unique colors
colormap = cm.linear.Set1_06.to_step(10).scale(0, 10)


for idx, coords in enumerate(trip_coords):
    color = colormap(idx)
    feature_group = folium.FeatureGroup(name=f'Trip {idx + 1}', tooltip=f"Line {idx+1}")
    [folium.CircleMarker(location=(lat, lon), radius=4, color=color, fill=True, fill_opacity=1).add_to(feature_group) for lon, lat in coords]
    folium.PolyLine([(lat, lon) for lon, lat in coords], color=color, weight=2).add_to(feature_group)
    fmap.add_child(feature_group, )

# Layer control and save map
fmap.add_child(folium.LayerControl())
fmap
# img_data = fmap._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save('task4.png')

## Visualisation using Folium for method 2

In [167]:
output = []
for top_osmid in method_2:
    way = get_way(top_osmid[0])
    output.append(way)

In [175]:
# Name of road semgents
[i['tags']['name'] for i in output]

['Campo dos Mártires da Pátria',
 'Rua de Damião de Góis',
 'Rua do Campo Alegre',
 'Rua da Constituição',
 'Rua de Saraiva de Carvalho',
 'Avenida de Camilo',
 'Rua do Passeio Alegre',
 'Rua do Freixo',
 'Rua de Egas Moniz',
 'Rua de Pinto Bessa']

In [228]:
trip_coords = []
for o in output:
    coords = [[xy['lon'], xy['lat']] for xy in o['geometry']]
    trip_coords.append(coords)

In [232]:
# Print out the coordinates to make sure same format as method 1 above
trip_coords[0][:4]

[[-8.617693, 41.1463677],
 [-8.6176798, 41.1463144],
 [-8.6173499, 41.1449765],
 [-8.6173405, 41.1449455]]

In [239]:
# Bounding box for map view
all_points = [point for trip in trip_coords for point in trip]
lons, lats = zip(*all_points)

# Set map boundaries with margin (consistent with Code 2)
margin = 0.005
lon_min, lon_max = min(lons) - margin, max(lons) + margin
lat_min, lat_max = min(lats) - margin, max(lats) + margin

# Calculate center for consistent centering across both codes
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

# Initialize map and set bounds
fmap = folium.Map(location=[center_lat, center_lon], zoom_start=13, width=1500, height=1000)
fmap.fit_bounds([[lat_min, lon_min], [lat_max, lon_max]])
# Plot trips with unique colors
colormap = cm.linear.Set1_06.to_step(10).scale(0, 10)


for idx, coords in enumerate(trip_coords):
    color = colormap(idx)
    feature_group = folium.FeatureGroup(name=f'Trip {idx + 1}', tooltip=f"Line {idx+1}")
    [folium.CircleMarker(location=(lat, lon), radius=4, color=color, fill=True, fill_opacity=1).add_to(feature_group) for lon, lat in coords]
    folium.PolyLine([(lat, lon) for lon, lat in coords], color=color, weight=2).add_to(feature_group)
    fmap.add_child(feature_group, )
    

# Layer control and save map
fmap.add_child(folium.LayerControl())
fmap
# img_data = fmap._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save('task4.png')